In [2]:
import keras
from tqdm import tqdm 
import tensorflow as tf 
from keras.applications.resnet50 import ResNet50
from keras.layers import Flatten, Input
from keras.models import Model
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
import numpy as np
#from google.cloud import storage 
from io import BytesIO
import time
import cv2
import keras
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Model, load_model
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.optimizers import SGD
start = time.time()


Using TensorFlow backend.
D:\Anaconda\envs\tensor_api\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Anaconda\envs\tensor_api\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Anaconda\envs\tensor_api\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Anaconda\envs\tensor_api\lib\site-packages\tensorflow\python\framewo

In [5]:
!pip install opencv-python

  Using cached opencv_python-4.2.0.32-cp37-cp37m-win_amd64.whl (33.0 MB)


In [13]:
train_path = "Train"
test_path = "Test"


batch_size = 24
image_size = 224
num_class = 4
# check_path = "gdrive/My Drive/GoogleColab/Project/Data/Check"

In [14]:
#Load the Resnet model
#base_model = VGG16(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))
b_model = ResNet50(weights='imagenet', pooling=max, include_top = False, input_shape=(image_size, image_size, 3) ) 


##print(base_model.summary())

    # Freeze the layers 
for layer in b_model.layers:
    layer.trainable = False
 
##Create the model
model = keras.models.Sequential()

# # Add the vgg convolutional base model
model.add(b_model)
 
# # Add new layers
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_class, activation='softmax'))
 
# # Show a summary of the model. Check the number of trainable parameters    
##print(model.summary())

In [15]:
from keras.optimizers import SGD

model.compile(loss='categorical_crossentropy',
          optimizer=SGD(lr=1e-3),
          metrics=['accuracy'])

train_datagen = ImageDataGenerator(
         zoom_range=0.2, # randomly zoom into images
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images
 
validation_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(image_size, image_size),
        batch_size=batch_size,
        class_mode='categorical',
         shuffle = True)
 
validation_generator = validation_datagen.flow_from_directory(
        test_path,
        target_size=(image_size, image_size),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True)

Found 2000 images belonging to 4 classes.
Found 400 images belonging to 4 classes.


In [16]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=100,
      validation_data= validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size)
 
# Save the model
model.save('resnet_model.h5')

Epoch 1/100
84/83 [==============================] - 24s 288ms/step - loss: 2.1061 - accuracy: 0.3815 - val_loss: 1.0692 - val_accuracy: 0.5400
Epoch 2/100
84/83 [==============================] - 20s 244ms/step - loss: 1.3822 - accuracy: 0.4970 - val_loss: 1.1260 - val_accuracy: 0.5400
Epoch 3/100
84/83 [==============================] - 22s 263ms/step - loss: 1.1714 - accuracy: 0.5445 - val_loss: 1.0425 - val_accuracy: 0.5750
Epoch 4/100
84/83 [==============================] - 23s 272ms/step - loss: 1.0689 - accuracy: 0.5860 - val_loss: 1.3174 - val_accuracy: 0.6200
Epoch 5/100
84/83 [==============================] - 23s 270ms/step - loss: 0.9658 - accuracy: 0.6330 - val_loss: 0.7967 - val_accuracy: 0.5850
Epoch 6/100
84/83 [==============================] - 23s 271ms/step - loss: 0.9340 - accuracy: 0.6330 - val_loss: 0.8947 - val_accuracy: 0.6175
Epoch 7/100
84/83 [==============================] - 21s 251ms/step - loss: 0.8790 - accuracy: 0.6565 - val_loss: 0.7801 - val_accuracy:

In [3]:
model = load_model("resnet_model.h5")

In [5]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.resnet import preprocess_input
from keras.applications.resnet import decode_predictions
import os
import heapq

model.compile(loss='categorical_crossentropy',
          optimizer=SGD(lr=1e-3),
          metrics=['accuracy'])


# def correct_path_function(path_name):
#     b =path_name.split("/")
#     del b[0]
#     delimiter = "/"
#     path = delimiter.join(b)
#     return path
  
# correct_path_function("content/gdrive/My Drive/GoogleColab/Project/Data/Check")
# path_of_test_files = path
# print(path_of_test_files)
  


path_of_test_files = "Tester"
list_of_test_images = os.listdir(path_of_test_files)

for i in list_of_test_images: 
  image_name, image_ext = os.path.splitext(i)
  img_path = os.path.join(path_of_test_files, i)
  img = load_img(img_path, target_size=(224, 224))
  x = img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)
  
  preds = model.predict(x)
  bruh = preds.flatten()
  predss= preds.tolist()
  duh = heapq.nlargest(3, range(len(bruh)), bruh.take) 
  y = os.listdir("Test")
  y.sort()
  print("Actual image: {}.....Predicted image: {},{}".format(image_name,y[duh[0]],y[duh[1]]))
  

Actual image: 639329-indian-men.....Predicted image: test_indian,test_white
Actual image: blackmwn.....Predicted image: test_black,test_asian
Actual image: chinese.....Predicted image: test_asian,test_white
Actual image: india.....Predicted image: test_indian,test_asian
Actual image: mohan.....Predicted image: test_asian,test_indian
Actual image: pexels-photo-774909-whitewomen.....Predicted image: test_indian,test_black
Actual image: ww.....Predicted image: test_white,test_asian
